## gather mutltiview samples

In [ ]:
from capture_utils import *

In [ ]:
# cv2.imshow('frame', cap.read()[1])
# cv2.imwrite('test.png', cap.read()[1])
# plt.imshow(cap.read()[1])
# plt.show()
# cv2.waitKey(0)




In [ ]:
display_drawer()
run_aruco_detector()


In [ ]:
photometric_calibration()

In [ ]:
import adv_optimization_utils as adv



In [ ]:
adv.optimize_patch()

In [ ]:
import torch
from classfier import *
res_dir = './results/'
# chose most recent result
import os
results = os.listdir(res_dir)
results = [r for r in results if 'working_patch_' in r]
results = sorted(results, key=lambda x: os.path.getmtime(os.path.join(res_dir, x)))
load_path = res_dir + results[-1]
print('loading from:', load_path)
all_patchs_results = os.listdir(res_dir)
all_patchs_results = [r for r in all_patchs_results if 'working_patchs_all_multiview' in r]
all_patchs_results = sorted(all_patchs_results, key=lambda x: os.path.getmtime(os.path.join(res_dir, x)))
load_path_all = res_dir + all_patchs_results[-1]
print('loading all from:', load_path_all)


In [ ]:
patchs = torch.load(load_path_all)#torch.load('./results/working_patchs_all_multiview_2025-09-18_00_20.pth')
tt = lambda x: torch.tensor(cv2.cvtColor(x, cv2.COLOR_BGR2RGB)/255.).permute(2,0,1).float()



In [ ]:
preds = []
found = False
good_patchs = []
for idx, pp in enumerate(patchs[1]):
    if 'jeep'  in patchs[0][idx] or 'half track' in patchs[0][idx]:
        continue

    plot_on_screen(pp.cpu().permute(1,2,0).numpy()*255)
    for i in range(10):
        r = cap.read()[1]
        r = cv2.cvtColor(r, cv2.COLOR_BGR2RGB)
        tr = tt(r)
        res= resnet_predict(tr.cuda().unsqueeze(0))
        # add text
        cv2.putText(r, f'Pred: {res}', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        cv2.imshow('frame', cv2.cvtColor(r, cv2.COLOR_BGR2RGB))
        preds.append(res)
        plt.imshow(r)
        plt.show()
        IPython.display.clear_output(wait=True)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if 'jeep' not in res and 'half track' not in res:
            found = True
            break
    if found:
        good_patchs.append(pp)
        

In [ ]:
plot_on_screen(good_patchs[0].cpu().permute(1,2,0).numpy()*255)

In [ ]:
# pred_pairs = list(zip(patchs[0], range(len(patchs[0]))))
# print(pred_pairs)
# for p in pred_pairs:
#     if p[0] != 'jeep' and p[0] != 'half track':       
#         print(p[0])
#         patch = patchs[1][p[1]]
#         break


In [ ]:


# patch = torch.load('./results/working_patch_multiview_2025-09-17_07:23.pth')
# tt = lambda x: torch.tensor(cv2.cvtColor(x, cv2.COLOR_BGR2RGB)/255.).permute(2,0,1).float()
# patch = torch.load(load_path)
# plot_on_screen(patch.cpu().permute(1,2,0).numpy()*255)

In [ ]:
caps = []
results = []
for i in range(1000):
    r = cap.read()[1]
    tr = tt(r)
    res= resnet_predict(tr.cuda().unsqueeze(0))
    print(res)
    # add text
    cv2.putText(r, f'Pred: {res}', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow('frame', r)
    results.append(res)

    caps.append(r)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [ ]:
video_name = 'captured_video_jeep2.avi'
height, width, layers = caps[0].shape
video = cv2.VideoWriter(video_name, 0, 30, (width,height))
for captured in caps:
    video.write(captured)

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

import glob
import tqdm
import matplotlib.pyplot as plt
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
import time
from tqdm import tqdm
import copy
from IPython.display import display, Image, clear_output
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import datetime


import cv2
import os
import glob


import kornia
import tqdm

import torchvision

tt = torchvision.transforms.ToTensor()

import cv2
import cv2.aruco as aruco
import numpy as np

from classfier import *

import pickle as pkl

with open("photometric_calibration.pkl", "rb") as f:
    data = pkl.load(f)

height = data['height']
width = data['width']

resizer = torchvision.transforms.Resize((height, width))
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Define parameters for ArUco marker detection
aruco_dict_type = cv2.aruco.DICT_6X6_250 # Change dictionary type if needed
marker_length = 0.05  # Marker length in meters (adjust as needed)
aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)

marker_id = 40
marker_size = 500  # Size in pixels
marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)


aruco_dict = aruco.getPredefinedDictionary(aruco_dict_type)
parameters = aruco.DetectorParameters()

# Detect ArUco markers
detector = aruco.ArucoDetector(aruco_dict, parameters)


from diffusers import StableDiffusionPipeline
import torch

# Load stable diffusion model


def decode_latents_grad(latents):
    # latents = F.interpolate(latents, (64, 64), mode='bilinear', align_corners=False)
    latents = 1 / 0.18215 * latents

    imgs = vae.decode(latents).sample

    imgs = (imgs / 2 + 0.5).clamp(0, 1)

    return imgs

def decode_latents(latents):
    # latents = F.interpolate(latents, (64, 64), mode='bilinear', align_corners=False)
    with torch.no_grad():
        with torch.amp.autocast(device):
            latents = 1 / 0.18215 * latents

            with torch.no_grad():
                imgs = vae.decode(latents).sample

            imgs = (imgs / 2 + 0.5).clamp(0, 1)

    return imgs

def encode_imgs(imgs):
    # imgs: [B, 3, H, W]
    with torch.no_grad():
        with torch.amp.autocast(device):
            imgs = 2 * imgs - 1

            posterior = vae.encode(imgs).latent_dist
            latents = posterior.sample() * 0.18215

    return latents



class framesDataset(Dataset):
    def __init__(self, frames, Hs):
        self.frames = frames
        self.Hs = Hs

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        H = self.Hs[idx]

        # Convert to tensor and normalize
        frame_tensor = tt(frame)

        return frame_tensor, H.astype(np.float32)
    





def warp(decoded_latents,H_t):
    dst_img_shape = valid_frames[0].shape[:2]
    warped_imgs = []
    for decoded_latent in decoded_latents:
        img = decoded_latent.unsqueeze(0).float().repeat(H_t.shape[0], 1, 1, 1)
        w=  kornia.geometry.transform.warp_perspective(img, H_t, dst_img_shape)
        warped_imgs.append(w)
    return torch.stack(warped_imgs, dim=0)#.squeeze(1)


vae = None
valid_frames = None



In [2]:
    
    os.makedirs('./results', exist_ok=True)
    curr_without_sec = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    curr_without_sec = curr_without_sec.replace(" ", "_").replace(":", "_")




    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")

    vae =  pipe.vae.to(device).eval()

    latent = (torch.rand((30,4, 4, 4), device=device) - 0.5) * 2

    # latent = torch.load(r"C:\git\PhysicalAdverserialProj\results\working_latent_multiview_2025-09-15_20").to(device)
    latent = latent.clone().detach()
    with torch.no_grad():
        decoded_latents = resizer(decode_latents(latent))


    l_size_h = decoded_latents.shape[-2]
    l_size_w = decoded_latents.shape[-1]

    orig_img_corners = np.array([[0,0],[l_size_w,0],[l_size_w,l_size_h],[0,l_size_h]], dtype=np.float32)


    border_size = 0

    ls = os.listdir('.')
    captures = [f for f in ls if f.startswith('captures_frames_multiview_') ]
    cap_dir = f'captures_frames_multiview_{len(captures)-1}'
    print('using capture dir', cap_dir)

    valid_frame_paths = glob.glob(f'{cap_dir}/*.png')

    valid_frames = []

    Hs = []

    for path in tqdm.tqdm_notebook(valid_frame_paths):
        img = cv2.imread(path)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        corners, ids, _ = detector.detectMarkers(gray)

        with torch.no_grad():
            pr = resnet_predict_raw(tt(img).cuda().unsqueeze(0))

        if ids is not None and pr.argmax(1) in orig_clases:

            c = corners[0][0]
            unbordred_corners = np.array([[c[0][0]-border_size, c[0][1]+border_size],
                                [c[1][0]-border_size, c[1][1]-border_size],
                                [c[2][0]+border_size, c[2][1]-border_size],
                                [c[3][0]+border_size, c[3][1]+border_size]])

            dst_pts = unbordred_corners
            H, _ = cv2.findHomography(orig_img_corners, dst_pts, cv2.RANSAC)

            Hs.append(H)
            valid_frames.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    print(f"Found {len(valid_frames)} valid frames with ArUco markers and original classes.")



    ds = framesDataset(valid_frames, Hs)


    print('creating dataloader')
    train,test = torch.utils.data.random_split(ds, [int(len(ds)*0.8), len(ds)-int(len(ds)*0.8)])
    train_loader = DataLoader(train, batch_size=1, shuffle=True, num_workers=0)
    test_loader = DataLoader(test, batch_size=10, shuffle=False, num_workers=0)

    print('dataloader created')
        
    latent.requires_grad = True


    latent_opt = torch.optim.Adam([latent], lr=0.1)


    latent.data = torch.load("./results/working_latent_multiview_2025-09-18_20_00.pth").to(device)
    


    jitter = T.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1)
    jitter_total_photo = T.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1)
    jitter_with_hue = T.ColorJitter(brightness=0.5,contrast=0.5,saturation=0.5,hue=0.1)

    augmentor = data['augmentor'].to(device).eval()
    mapper = lambda x: jitter(augmentor(x))
    # mapper = lambda x: jitter(x)

    print('starting optimization')
    i = 0 
    # blend_ratio = 1
    for epoch in range(50):
        for frames, H_t in tqdm.tqdm(train_loader):

            i += 1
            # adv_loss = lbfgs_latent_opt.step(latent_closure_adp)

            frames = frames.to('cuda')

            blend_ratio = torch.rand(1).cuda() * 0.2 + 0.8
            stronger_aug_steps_thresh = 400
            
            if i == stronger_aug_steps_thresh:
                print('using stronger augmentations')

            if i < stronger_aug_steps_thresh:
                mapper = lambda x: jitter(x)

            else:
                mapper = lambda x: jitter(torch.stack([augmentor(xx).to(device) for xx in x]))

            latent_opt.zero_grad()
            adv_patch = resizer(decode_latents_grad(latent).float())

            adv_patch_m = mapper(adv_patch)

            # w_mask  =warp(adv_patch_m*0+1)
            # w  =warp(adv_patch_m)
            w_mask  =warp(adv_patch_m*0+1, H_t.cuda())
            w  = warp(adv_patch_m, H_t.cuda())

            sum_tensor = ((w_mask != 0) * -blend_ratio + 1) * frames + w *blend_ratio

            sum_tensor = sum_tensor.view(sum_tensor.shape[0]*sum_tensor.shape[1], sum_tensor.shape[2], sum_tensor.shape[3], sum_tensor.shape[4])

            sum_tensor = jitter_total_photo(sum_tensor)

            adv_loss = adv_loss_calc2(sum_tensor).mean() #+ adv_patch.norm() / 10000
            
            adv_loss.backward()

            if i % 1 == 0:
                latent_opt.step()

            if i % 10 == 0:
                unsuccessful_patches_idxs = torch.arange(0, sum_tensor.shape[0])[torch.tensor([x in orig_clases for x in resnet_predict_raw(sum_tensor).argmax(1)])]
                successful_patches_idxs = torch.arange(0, sum_tensor.shape[0])[torch.tensor([x not in orig_clases for x in resnet_predict_raw(sum_tensor).argmax(1)])]
                for idx in unsuccessful_patches_idxs:
                    if successful_patches_idxs.shape[0] != 0:
                        rand_succ_idx = successful_patches_idxs[torch.randint(0, successful_patches_idxs.shape[0], (1,)).item()]
                        latent.data[idx] = latent.data[rand_succ_idx] + (torch.randn_like(latent.data[rand_succ_idx]) * 0.05)
                    else:
                        latent.data[idx] = (torch.rand((4, 4, 4), device=device) - 0.5) * 2


            if i % 200 == 0:
                # print(f"Iteration {i}, Loss: {latent_closure_adp().item():.4f}")
                print(f"Iteration {i}, Loss: {adv_loss.item():.4f}. grad_norm: {latent.grad.norm().item():.4f}")

                with torch.no_grad():

                    for frames, H_t in tqdm.tqdm(test_loader):

                        adv_patch = resizer(decode_latents(latent).float())

                        frames = frames.to('cuda')

                        
                        adv_patch_m = mapper(adv_patch)

                        # w_mask  =warp(adv_patch_m*0+1)
                        # w  =warp(adv_patch_m)
                        w_mask  =warp(adv_patch_m*0+1, H_t.cuda())
                        w  = warp(adv_patch_m, H_t.cuda())

                        sum_tensor = ((w_mask != 0) * -blend_ratio + 1) * frames + w *blend_ratio

                        sum_tensor = sum_tensor.view(sum_tensor.shape[0]*sum_tensor.shape[1], sum_tensor.shape[2], sum_tensor.shape[3], sum_tensor.shape[4])

                        sum_tensor = jitter_total_photo(sum_tensor)

                        adv_loss = adv_loss_calc2(sum_tensor)

                        sum_tensor = sum_tensor.cpu()
                        argmin = torch.argmin(adv_loss).cpu().item()

                        plt.imshow(sum_tensor[argmin].permute(1,2,0).numpy())
                        plt.show()
                        print(resnet_predict(sum_tensor[argmin].unsqueeze(0).cuda()))

                        test_pred = resnet_predict(sum_tensor.cuda())
                        if type(test_pred) == list:
                            pred = resnet_predict(sum_tensor.cuda())
                            print(np.unique(pred,return_counts=True))
                        # print(lx)

                        out_path =  rf'./results/working_latent_multiview_{curr_without_sec}.pth'
                        torch.save(latent.cpu().detach(), out_path)
                        print('saved latent to', out_path)
                        out_path =  rf'./results/working_patch_multiview_{curr_without_sec}.pth'
                        torch.save(adv_patch[argmin// sum_tensor.shape[0]].cpu().detach(), out_path)
                        print('saved patch to', out_path)
                        out_path =  rf'./results/working_patchs_all_multiview_{curr_without_sec}.pth'
                        torch.save((pred,adv_patch.cpu().detach()), out_path)
                        print('saved patch to', out_path)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

using capture dir captures_frames_multiview_16


C:\Users\danny\AppData\Local\Temp\ipykernel_50544\1026747868.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm.tqdm_notebook(valid_frame_paths):


  0%|          | 0/401 [00:00<?, ?it/s]

Found 122 valid frames with ArUco markers and original classes.
creating dataloader
dataloader created
starting optimization


 19%|█▊        | 18/97 [00:08<00:35,  2.20it/s]


KeyboardInterrupt: 

In [3]:
latent

tensor([[[[-0.6226, -1.8115,  1.7012,  0.0859],
          [-0.1678, -2.8223, -4.7617,  1.4131],
          [-0.9912, -3.6777, -0.4551, -1.4316],
          [ 0.0331, -4.0977, -0.1975, -2.1953]],

         [[-4.1055, -4.5781, -3.1504, -1.6455],
          [-0.5840, -0.8682,  2.4238, -1.5693],
          [-1.7598, -5.1445, -3.3164, -0.4890],
          [ 0.2874, -0.6660, -2.7363, -0.6973]],

         [[-2.8086, -0.1630,  2.9395, -2.4727],
          [-2.1875,  0.4172, -1.1025, -1.7402],
          [-1.5420,  2.6367, -0.0565, -2.4863],
          [-0.6499,  0.1631, -0.1801,  1.1807]],

         [[ 0.8496,  0.4580,  2.5488,  3.1641],
          [ 0.8945,  0.7461,  1.5439,  1.8877],
          [-0.2379,  1.1299, -1.4082,  1.8057],
          [ 2.2266, -1.5205,  2.7227,  1.3740]]],


        [[[ 1.7012,  0.6582, -0.0848,  1.0977],
          [-0.0061,  2.2129,  2.0469, -0.6880],
          [-2.2402, -1.1943, -1.1074, -0.6025],
          [-3.3633, -1.5498, -2.6660, -1.1260]],

         [[-1.9707, -0.5518,

In [ ]:
latent

In [ ]:
for i, c in enumerate(weights.meta["categories"]):
    print(i, c)